In [1]:
import datetime
import os
import time
import random

import numpy as np
import tensorflow as tf

from matplotlib import pyplot as plt
from tensorflow.keras import backend as K
from tensorflow.keras.layers import (Activation, BatchNormalization, Dense,
                                     Dropout, Flatten, Input, Reshape,
                                     TimeDistributed, Lambda)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.losses import binary_crossentropy

import keras

tf.compat.v1.disable_eager_execution()

In [8]:
%load_ext tensorboard

In [8]:
import midi

In [9]:
NUM_EPOCHS = 50
LR = 0.001
WRITE_HISTORY = True
NUM_RAND_SONGS = 10
DO_RATE = 0.1
BN_M = 0.9

BATCH_SIZE = 256
MAX_LENGTH = 16

np.random.seed(0)
random.seed(0)

In [10]:
# y_samples = np.load('samples.npy')
# y_lengths = np.load('lengths.npy')
y_samples = np.load('samplesBach.npy')
y_lengths = np.load('lengthsBach.npy')

num_songs = y_lengths.shape[0]

y_shape = (num_songs, MAX_LENGTH) + y_samples.shape[1:]
y_orig = np.zeros(y_shape, dtype=y_samples.dtype)

In [11]:
cur_ix = 0
for i in range(num_songs):
    end_ix = cur_ix + y_lengths[i]
    for j in range(MAX_LENGTH):
        k = j % (end_ix - cur_ix) 
        y_orig[i,j] = y_samples[cur_ix + k]
    cur_ix = end_ix

In [12]:
y = np.copy(y_orig)

y_train = y[:178]
y_valid = y[178:238]

y_test_song = np.copy(y[0])

In [13]:
class Sampling(keras.layers.Layer):
    def call(self, inputs):
        mean, log_var = inputs 
        return K.random_normal(tf.shape(log_var), stddev=0.1) * K.exp(log_var / 2) + mean


In [10]:
# PARAM_SIZE = 120

# x_in = Input(shape=y_shape[1:])
# z = Reshape((y_shape[1], -1))(x_in)
# z = TimeDistributed(Dense(2000, activation='relu'))(z)
# z = TimeDistributed(Dense(200, activation='relu'))(z)
# z = Flatten()(z)
# z = Dense(1600, activation='relu')(z)

# codings_mean = keras.layers.Dense(PARAM_SIZE)(z) # μ
# codings_log_var = keras.layers.Dense(PARAM_SIZE)(z) # γ
# codings = Sampling()([codings_mean, codings_log_var])
# variational_encoder = keras.Model(inputs=[x_in], outputs=[codings_mean, codings_log_var, codings])

# decoder_inputs = keras.layers.Input(shape=[PARAM_SIZE])
# x = Dense(1600, name='encoder')(decoder_inputs)
# x = BatchNormalization(momentum=BN_M)(x)
# x = Activation('relu')(x)
# x = Dropout(DO_RATE)(x)
# x = Dense(MAX_LENGTH * 200)(x)
# x = Reshape((MAX_LENGTH, 200))(x)
# x = TimeDistributed(BatchNormalization(momentum=BN_M))(x)
# x = Activation('relu')(x)
# x = Dropout(DO_RATE)(x)
# x = TimeDistributed(Dense(2000))(x)
# x = TimeDistributed(BatchNormalization(momentum=BN_M))(x)
# x = Activation('relu')(x)
# x = Dropout(DO_RATE)(x)
# x = TimeDistributed(Dense(y_shape[2] * y_shape[3], activation='sigmoid'))(x)
# outputs = Reshape((y_shape[1], y_shape[2], y_shape[3]))(x)

# variational_decoder = keras.Model(inputs=[decoder_inputs], outputs=[outputs])
# _, _, codings = variational_encoder(x_in)
# reconstructions = variational_decoder(codings)
# variational_ae = keras.Model(inputs=[x_in], outputs=[reconstructions])

# latent_loss = -0.5 * K.sum(
# 1 + codings_log_var - K.exp(codings_log_var) - K.square(codings_mean),
# axis=-1)

# # latent_loss = 0.02 * K.mean(1 + codings_log_var - K.square(codings_mean) - K.exp(codings_log_var), axis=None)

# variational_ae.add_loss(K.mean(latent_loss) / 147456.)
# variational_ae.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [14]:
PARAM_SIZE = 16

x_in = Input(shape=y_shape[1:])
z = Reshape((y_shape[1], -1))(x_in)
z = TimeDistributed(Dense(64, activation="relu"))(z)
z = Flatten()(z)
z = Dense(1024, activation="relu")(z)
z = Dense(256, activation="relu")(z)
z = Dense(64, activation="relu")(z)

codings_mean = keras.layers.Dense(PARAM_SIZE)(z)  # μ
codings_log_var = keras.layers.Dense(PARAM_SIZE)(z)  # γ
codings = Sampling()([codings_mean, codings_log_var])
variational_encoder = keras.Model(
    inputs=[x_in], outputs=[codings_mean, codings_log_var, codings]
)


decoder_inputs = keras.layers.Input(shape=[PARAM_SIZE])
x = Dense(64, name="encoder")(decoder_inputs)
x = BatchNormalization(momentum=BN_M)(x)
x = Activation("relu")(x)
x = Dropout(DO_RATE)(x)
x = Dense(256)(x)
x = Dense(1024)(x)
x = Reshape((16, 64))(x)
x = TimeDistributed(BatchNormalization(momentum=BN_M))(x)
x = Activation("relu")(x)
x = Dropout(DO_RATE)(x)
x = TimeDistributed(Dense(9216))(x)
x = TimeDistributed(BatchNormalization(momentum=BN_M))(x)
x = Activation("relu")(x)
x = Dropout(DO_RATE)(x)
x = TimeDistributed(Dense(y_shape[2] * y_shape[3], activation="sigmoid"))(x)
outputs = Reshape((y_shape[1], y_shape[2], y_shape[3]))(x)

variational_decoder = keras.Model(inputs=[decoder_inputs], outputs=[outputs])

_, _, codings = variational_encoder(x_in)
reconstructions = variational_decoder(codings)
variational_ae = keras.Model(inputs=[x_in], outputs=[reconstructions])

latent_loss = -0.5 * K.sum(
    1 + codings_log_var - K.exp(codings_log_var) - K.square(codings_mean), axis=-1
)

variational_ae.add_loss(K.mean(latent_loss) / 147456.0)
variational_ae.compile(loss="binary_crossentropy", optimizer="rmsprop")


Instructions for updating:
Colocations handled automatically by placer.


InternalError: cudaGetDevice() failed. Status: cudaGetErrorString symbol not found.

In [12]:
log_dir = os.path.join(
    "logs",
    datetime.datetime.now().strftime("%Y%m%d-%H%M%S"),
)
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir=log_dir),
]

In [13]:
history = variational_ae.fit(
    y_train,
    y_train,
    epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(y_valid, y_valid),
    callbacks=callbacks,
    verbose=2,
)

loss = history.history["loss"][-1]
print(f"Train Loss: {loss}")

write_dir = 'HistoryVAAuto/'
variational_ae.save('HistoryVAAuto/variational_ae.h5')

Train on 178 samples, validate on 60 samples
Epoch 1/50


c:\Users\eller\Desktop\iniciaçãoCientifica\music-generation\env\lib\site-packages\keras\engine\training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


178/178 - 10s - loss: 0.7480 - val_loss: 0.6703 - 10s/epoch - 55ms/sample
Epoch 2/50


In [ ]:
teste = np.random.normal(0.0, 1.0, (PARAM_SIZE))
y_song = variational_decoder.predict(teste.reshape(1,PARAM_SIZE))
midi.samples_to_midi(y_song[0], write_dir + 'test1.mid', 96, 0.5)

c:\Users\eller\Desktop\iniciaçãoCientifica\music-generation\env\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [ ]:
func = K.function([variational_decoder.get_layer('encoder').input, K.learning_phase()], [variational_decoder.layers[-1].output])
enc = Model(inputs=variational_encoder.input, outputs=variational_encoder.layers[-1].output)

In [ ]:
rand_vecs = np.random.normal(0.0, 1.0, (NUM_RAND_SONGS, PARAM_SIZE))
np.save('rand.npy', rand_vecs)

In [ ]:
def make_rand_songs(write_dir, rand_vecs):
    for i in range(rand_vecs.shape[0]):
        x_rand = rand_vecs[i:i+1]
        y_song = variational_decoder.predict(x_rand.reshape(1,PARAM_SIZE))
        midi.samples_to_midi(y_song[0], write_dir + 'rand' + str(i) + '.mid', 16, 0.5)

def make_rand_songs_normalized(write_dir, rand_vecs):
    x_enc = np.squeeze(enc.predict(y_orig))

    x_mean = np.mean(x_enc, axis=0)
    x_cov = np.cov((x_enc - x_mean).T)
    _, s, v = np.linalg.svd(x_cov)
    e = np.sqrt(s)

    print(f"Means: {x_mean[:6]}")
    print(f"Evals: {e[:6]} ")

    x_vecs = x_mean + np.dot(rand_vecs * e, v)
    make_rand_songs(write_dir, x_vecs)

In [ ]:
y_song = variational_ae.predict((y_test_song).reshape(1,16,96,96), batch_size=BATCH_SIZE)

midi.samples_to_midi(y_song[0], write_dir + 'test.mid', 16)
make_rand_songs_normalized(write_dir, rand_vecs)

Means: [ 0.2553613  -0.6570139  -0.30995566 -0.15215094  0.43708682  0.33621833]
Evals: [1.29500311 0.71246686 0.39803132 0.34032681 0.23375806 0.1849534 ] 


In [ ]:
# new_model = tf.keras.models.load_model('HistoryVAAuto/model.h5')